<a href="https://colab.research.google.com/github/noahlim99/MAIMO/blob/main/MAIMO_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytesseract

from google.colab import drive  # Import to mount Google Drive
from openai import OpenAI  # Ensure openai==1.52.2 or compatible
import os
from PIL import Image
import pytesseract
import re

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install -y tesseract-ocr-kor
!pip install pytesseract

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,110 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,487 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Get:13 http://archive.ubuntu.com/u

In [8]:
# 이미지에서 텍스트 추출 및 저장
def extract_and_save_text(input_dir, output_dir, lang='kor+eng'):
    """
    Google Drive에 저장된 이미지를 텍스트로 변환하여 저장.

    Parameters:
        input_dir (str): 이미지 파일이 저장된 입력 디렉토리 (Google Drive)
        output_dir (str): 추출된 텍스트 파일을 저장할 출력 디렉토리 (Google Drive)
        lang (str): Tesseract OCR 언어 설정 (기본값: 'kor+eng')
    """
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # 이미지 파일 처리
    for img_path in os.listdir(input_dir):
        if img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            try:
                input_path = os.path.join(input_dir, img_path)
                output_path = os.path.join(output_dir, os.path.splitext(img_path)[0] + '.txt')

                # 이미지에서 텍스트 추출
                text = pytesseract.image_to_string(Image.open(input_path), lang=lang)

                # 한글, 영어, 숫자, 기호 필터링
                filtered_text = re.sub(r'[^\w\s가-힣A-Za-z0-9+\-*/=<>.,]', '', text)

                # 텍스트 파일로 저장
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(filtered_text)

                print(f"텍스트가 저장되었습니다: {output_path}")
            except Exception as e:
                print(f"이미지 처리 중 오류가 발생했습니다: {img_path}, 오류 메시지: {e}")

# 4. 입력 및 출력 디렉토리 경로 설정
input_dir = '/content/drive/My Drive/AIDS_Project/Images'  # 이미지 폴더 경로
output_dir = '/content/drive/My Drive/AIDS_Project/texts'  # 텍스트 저장 폴더 경로

# 5. 텍스트 추출 및 저장 함수 호출
extract_and_save_text(input_dir, output_dir)

텍스트가 저장되었습니다: /content/drive/My Drive/AIDS_Project/texts/CC4AE15E-96F8-418D-97D3-819D7AF1E841_1_201_a.txt


In [9]:
# Solar Pro API configuration
client = OpenAI(
    api_key="up_QlPWg5KR8ix0jWZnOBqsFjSAyltL1",
    base_url="https://api.upstage.ai/v1/solar"
)

In [18]:
def classify_subject_with_choices(content, subjects):
    """
    Classify the text into one of the subjects provided by the user.
    """
    subject_list = ", ".join(subjects)
    prompt = f"""
    Based on the given text, classify it into one of the following subjects:
    {subject_list}.
    Only return the name of the subject. Do not add extra explanations or context.
    - Text: "{content}"
    """

    response = client.chat.completions.create(
        model="solar-pro",
        messages=[{"role": "user", "content": prompt}],
        stream=False,
    )
    return response.choices[0].message.content.strip()

def summarize_and_expand(content):

    """
    Summarize the given content, focusing only on subject-related information, and then, provide directly relevant, in-depth details.
    """
    prompt = f"""
    Summarize the following text, focusing exclusively on the subject-related information, preserving its original meaning and structure.
    - Do not add new information or remove important details.
    - Remove any dates, names, locations, or unrelated details.
    - Make the summary concise and focused only on the core subject information.
    - Then, if applicable, provide "Additional Information:" with detailed, professional-level context or background about the subject matter itself.
    - Avoid explaining the summary itself. Instead, offer new, relevant insights directly related to the subject.
    - Text: "{content}"
    """

    response = client.chat.completions.create(
        model="solar-pro",
        messages=[{"role": "user", "content": prompt}],
        stream=False,
    )
    output = response.choices[0].message.content.strip()

    # Separate "Summary" and "Additional Information"
    summary, additional_information = "", ""
    if "Additional Information:" in output:
        parts = output.split("Additional Information:", 1)
        summary = parts[0].strip().removeprefix("Summary:").strip()
        additional_information = parts[1].strip()
    else:
        summary = output.strip().removeprefix("Summary:").strip()

    return summary, additional_information

def main():
    # Step 1: Get the list of subjects from the user
    print("수강 중인 수업을 입력해 주세요.(콤마로 구분) (예: Mathematical Statistics, Intermediate Financial Accounting):")
    user_input = input("Subjects: ").strip()
    subjects = [subject.strip() for subject in user_input.split(",")]

    if not subjects:
        print("수업 리스트가 비어 있습니다. 프로그램을 종료합니다.")
        return

    print(f"입력된 수업 목록: {subjects}")
    print("-" * 50)

    # Step 2: Define the text files path in Google Drive
    texts_dir = "/content/drive/My Drive/AIDS_Project/texts"  # 텍스트 파일이 저장된 디렉토리

    # Step 3: Process all text files in the directory
    try:
        for txt_file in os.listdir(texts_dir):
            if txt_file.endswith('.txt'):
                file_path = os.path.join(texts_dir, txt_file)

                # Step 3.1: Read the content of the text file
                with open(file_path, "r", encoding="utf-8") as file:
                    content = file.read()

                # Step 3.2: Classify the text into a subject
                classified_subject = classify_subject_with_choices(content, subjects)

                # Step 3.3: Summarize and expand the content
                summary, additional_information = summarize_and_expand(content)

                # Step 3.4: Display the results
                print(f"파일 경로: {file_path}")
                print(f"Classified Subject: {classified_subject}")
                print("\nSummary:")
                print(summary)
                if additional_information:
                    print("\nSuggested additional information:")
                    print(additional_information)
                print("-" * 50)

    except Exception as e:
        print(f"텍스트 파일 처리 중 오류 발생: {txt_file} ({e})")

if __name__ == "__main__":
    main()

수강 중인 수업을 입력해 주세요.(콤마로 구분) (예: Mathematical Statistics, Intermediate Financial Accounting):
Subjects: 금융경제학
입력된 수업 목록: ['금융경제학']
--------------------------------------------------
파일 경로: /content/drive/My Drive/AIDS_Project/texts/CC4AE15E-96F8-418D-97D3-819D7AF1E841_1_201_a.txt
분류된 과목: 금융경제학

Summary:
The lecture discusses different types of investment companies, focusing on hedge funds. Hedge funds, like mutual funds, pool investors' money for collective investment but are not subject to many regulations. They can be categorized by risk classification into short-term, market directional (high risk), moderate risk funds (market neutral or value orientation), and absolute return funds (moderate risk-avoidance).

Suggested additional information:
Hedge funds are managed by professional investment managers who employ diverse and complex strategies, including short selling, leverage, and derivatives, to generate high returns. They typically require high minimum investments and are only acc

In [7]:
def delete_files_in_folder(folder_path):
    """
    Google Drive의 특정 폴더에 있는 모든 파일을 삭제.

    Parameters:
        folder_path (str): 삭제할 파일이 저장된 폴더 경로
    """
    try:
        if os.path.exists(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if os.path.isfile(file_path):  # 파일만 삭제
                    os.remove(file_path)
                    print(f"삭제 완료: {file_path}")
            print(f"{folder_path} 폴더가 초기화되었습니다.")
        else:
            print(f"{folder_path} 경로가 존재하지 않습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

# 이미지 및 텍스트 폴더 초기화
images_dir = "/content/drive/My Drive/AIDS_Project/Images"
texts_dir = "/content/drive/My Drive/AIDS_Project/texts"

print("이미지 폴더 초기화 중...")
delete_files_in_folder(images_dir)

print("텍스트 폴더 초기화 중...")
delete_files_in_folder(texts_dir)

#Reset Solar API
def reset_solar_api(api_key, base_url="https://api.upstage.ai/v1/solar"):
    """
    Solar API 초기화 함수.

    Parameters:
        api_key (str): Solar API 키
        base_url (str): Solar API 기본 URL (기본값: Solar API의 base URL)
    Returns:
        OpenAI: 초기화된 OpenAI 클라이언트
    """
    try:
        # 새로운 클라이언트 초기화
        client = OpenAI(api_key="up_QlPWg5KR8ix0jWZnOBqsFjSAyltL1", base_url="https://api.upstage.ai/v1/solar")
        print("Solar API가 초기화되었습니다.")
        return client
    except Exception as e:
        print(f"Solar API 초기화 중 오류 발생: {e}")
        return None

# Solar API 초기화 호출
api_key = "your_api_key_here"  # Solar API 키를 여기에 입력
client = reset_solar_api(api_key)

이미지 폴더 초기화 중...
삭제 완료: /content/drive/My Drive/AIDS_Project/Images/CC4AE15E-96F8-418D-97D3-819D7AF1E841_1_201_a.jpeg
/content/drive/My Drive/AIDS_Project/Images 폴더가 초기화되었습니다.
텍스트 폴더 초기화 중...
삭제 완료: /content/drive/My Drive/AIDS_Project/texts/CC4AE15E-96F8-418D-97D3-819D7AF1E841_1_201_a.txt
/content/drive/My Drive/AIDS_Project/texts 폴더가 초기화되었습니다.
Solar API가 초기화되었습니다.
